In [1]:
import requests
from selenium import webdriver
import base64
import re
from PIL import Image
import pandas as pd
import urllib
from io import BytesIO
from bs4 import BeautifulSoup as bs
import numpy as np
import cv2

In [10]:
navegador = webdriver.Firefox(keep_alive = False)
navegador.get('http://rais.gov.br/sitio/consulta_trabalhador_identificacao.jsf')
navegador.set_window_size(1920,1080)
for i in range(10):
    navegador.find_element_by_xpath('//*[@id="btnRecarregar_captcha_serpro_gov_br"]').click()
    #img = navegador.find_element_by_xpath('//*[@id="img_captcha_serpro_gov_br"]').screenshot(f'bdcaptcha/img{i}.png')
    img = navegador.find_element_by_xpath('//*[@id="img_captcha_serpro_gov_br"]').get_attribute('src').split(',')[1]
    img=base64.b64encode(base64.b64decode(img))
    fh = open(f"imagem{i}.png", "wb")
    fh.write(base64.decodebytes(img))
    fh.close()
    imagem = cv2.imread(f"imagem{i}.png",1)
    img_scale_up = cv2.resize(imagem, (0, 0), fx=3, fy=3)
    cv2.imwrite(f'imagem{i}.png',img_scale_up)


/tmp/ipykernel_33815/2326604045.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  navegador.find_element_by_xpath('//*[@id="btnRecarregar_captcha_serpro_gov_br"]').click()
/tmp/ipykernel_33815/2326604045.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  img = navegador.find_element_by_xpath('//*[@id="img_captcha_serpro_gov_br"]').get_attribute('src').split(',')[1]


In [6]:
imagem = cv2.imread('imageToSave.png',1)


img_scale_up = cv2.resize(imagem, (0, 0), fx=2.5, fy=2.5)
cv2.imwrite('imageToSave.png',img_scale_up)

True

In [49]:
def getToken(home='http://rais.gov.br'):
    response = requests.get(
        f'{home}/sitio/consulta_trabalhador_identificacao.jsf',
        headers={'Accept': "text/html",
                 "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0",

                 }
    )

    dados = response.text
    TOKEN = re.search('data-clienteid="(.*)">', dados).group(1).split('"')[0]
    cookiesx = response.cookies
    return TOKEN, cookiesx

In [250]:
def getData(home='http://rais.gov.br/sitio/consulta_trabalhador_identificacao.jsf', ini=0, qtd=40):
    TOKEN, cookiesx = getToken(home)
    cookie = "=".join(cookiesx.items()[0])
    headers = {
        'Cookie': cookie,
        'Origin': home,
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0',
        'Content-Type': 'application/json; charset=UTF-8',
        'Referer': home + '/sitio/consulta_trabalhador_identificacao.jsf',
        'Connection': 'keep-alive'
    }
    # params = {'sEcho': 1, 'iColumns': 5, 'sColumns': "", 'iDisplayStart': ini, 'iDisplayLength': qtd, 'mDataProp_0': 0,
    #           'mDataProp_1': 1, 'mDataProp_2': 2, 'mDataProp_3': 3, 'mDataProp_4': 4, 'sSearch': '', 'iSortCol_0': 1,
    #           'sSortDir_0': "desc", 'iSortingCols': 1, 'bSortable_0': 'true', 'bSortable_1': 'true',
    #           'bSortable_2': 'true', 'bSortable_3': 'true', 'bSortable_4': 'false', 'option': "com_mpt", 'task': "tacs",
    #           'format': "raw", TOKEN: 1}

    response2 = requests.post(
        f'{home}',
         headers=headers#,
        # data=params
    )
    try:
        json_data = response2.json()
        df_aadata = pd.DataFrame(json_data.get('aaData'))
        df_aadata.columns = ["cidade", "data", "numero", "procedimento", "arquivo"]
        df_aadata['path'] = [f'{home}/servicos/termos-de-ajuste-de-conduta?task=baixa&format=raw&arq={item}' for item in
                             df_aadata.arquivo]
        df_aadata['data'] = pd.to_datetime(df_aadata.data, format='%d/%m/%Y')
        df_aadata['dia_semana'] = df_aadata.data.dt.dayofweek
        df_aadata['id_lead'] = df_aadata[['cidade', 'numero', 'procedimento']].agg('_'.join, axis=1)

        return df_aadata
    except:
        print("não é json")
        return response2

In [251]:
a = getData()

não é json


In [252]:
a

<Response [200]>